In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {"movie": "Top Gun", "emojis": "🛩️👨‍✈️🔥"},
    {"movie": "The Godfather", "emojis": "👨‍👨‍👦🔫🍝"},
    {"movie": "Titanic", "emojis": "🚢💑❄️"},
]

example_prompt = PromptTemplate(
    input_variables=["movie", "emojis"],
    template="Movie: {movie}\nEmojis: {emojis}"
)

fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Movie: {input}\nEmojis:",
    input_variables=["input"]
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an emoji expert."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

def load_history(_):
    return memory.load_memory_variables({})["history"]

chain = (
    RunnablePassthrough.assign(history=load_history) |
    chat_prompt |
    RunnablePassthrough.assign(input=lambda d: {"input": d["messages"][-1].content}) |
    fewshot_prompt
)

def run_chain(user_input):
    prompt_text = chain.invoke({"input": user_input})
    memory.save_context({"input": user_input}, {"output": prompt_text})
    print(f"\n📩 질문: {user_input}\n📝 프롬프트 결과:\n{prompt_text}\n")

run_chain("Top Gun")
run_chain("The Godfather")
run_chain("What movie did I ask about first?")


AssertionError: The input to RunnablePassthrough.assign() must be a dict.